In [10]:
import math
import numpy as np

# Constants
A0 = 333  # Administered activity (MBq)
T = 1  # Occupancy factor1
T1_2 = 109.8 / 60  # Radionuclide half-life in hours (109.8 minutes)
dr = 0.092  # Dose rate immediately after administration (µSv m²/MBq h)
P_controlled = 100  # Weekly dose limit for controlled areas (µSv)
P_uncontrolled = 20  # Weekly dose limit for uncontrolled areas (µSv)

# Function to get user input for variables
def get_inputs():
    Nw = int(input("Enter Number of patients per week: "))
    d = float(input("Enter Distance from source to barrier (m): "))
    tU = float(input("Enter Uptake time (hours): "))
    tI = float(input("Enter Scan time (hours): "))
    print(f"Input values - Nw: {Nw}, d: {d}, tU: {tU}, tI: {tI}")
    return Nw, d, tU, tI



# Function to calculate the dose reduction factor over imaging time
def RtI(T1_2, tI):
    rti_value = 1.443 * (T1_2 / tI) * (1 - math.exp(-0.693 * tI / T1_2))
    print(f"RtI: {rti_value}")
    return rti_value



def Fu(tU):
    return math.exp (-0.693 * (tU*60) /110)

# Function to calculate transmission factor for the Scan room
def B_scan_room(P, d, T, Nw, A0, tI, Fu, RtI):
    return 12.8 * P * (d ** 2) / (T * Nw * A0 * 0.85 * tI * Fu* RtI)




# Main function to run the calculation for both controlled and uncontrolled areas
def calculate_transmission_factors():
    # Get user inputs
    Nw, d, tU, tI = get_inputs()
    
    # Calculate the dose reduction factor RtU & RtI
    rti = RtI(T1_2, tI)

    # Calculate the Fu
    fu = Fu(tU)

    # Calculate the transmission factor B for uncontrolled and controlled areas
    B_uc = B_scan_room(P_uncontrolled,d, T, Nw, A0, tI, fu, rti)
    B_c = B_scan_room(P_controlled, d, T, Nw, A0, tI, fu, rti)

    
     # Output the results
    print()  # This adds a blank line before the following prints
    print("****SCAN ROOM****")
    print(f"\nTransmission Factor (B) for the Scan room (Uncontrolled area): {B_uc:.4f}")
    print(f"Transmission Factor (B) for the Scan room (Controlled area): {B_c:.4f} ")

    return {

        "Scan Room Uncontrolled": B_uc,
        "Scan Room Controlled": B_c
    }

# Run the calculations and store the results
B_results = calculate_transmission_factors()


# X = Shileding Thickness
# Lead Parameters
lead_params = (1.543, -0.4408, 2.136)  # (alpha, beta, gamma)
# concrete Parameters
concrete_params = (0.1539, -0.1161, 2.0752)  # (alpha, beta, gamma)

def calculate_x(alpha, beta,  gamma, B):
    if B >= 1:
        # If B is greater than or equal to 1, no additional shielding is needed
        print("Since B is greater than or equal to 1, no additional shielding is required.")
        return 0
    # The Monte Carlo transmission data have been fitted  to the Model By the following parameters by Archer et al. 
    # #visit: https://pmc.ncbi.nlm.nih.gov/articles/PMC3155576/
    numerator = (B ** (-gamma)) + (beta / alpha)
    denominator = 1 + (beta / alpha)
    x = (1 / (alpha * gamma)) * math.log(numerator / denominator)
    return x

# Calculate thickness for uncontrolled area
B_uc = B_results["Scan Room Uncontrolled"]
lead_thickness_uc = calculate_x(*lead_params, B_uc)
concrete_thickness_uc = calculate_x(*concrete_params, B_uc)

# Calculate thickness for controlled area
B_c = B_results["Scan Room Controlled"]
lead_thickness_c = calculate_x(*lead_params, B_c)
concrete_thickness_c = calculate_x(*concrete_params, B_c)

# Output the thickness results
print()
print("****UnControl****")
print(f"Shielding thickness for lead (Uncontrolled Area): {lead_thickness_uc / 0.1:.2f} mm")
print(f"Shielding thickness for concrete (Uncontrolled Area): {concrete_thickness_uc:.2f} cm")
print()
print("****Control****")
print(f"Shielding thickness for lead (Controlled Area): {lead_thickness_c / 0.1:.2f} mm")
print(f"Shielding thickness for concrete (Controlled Area): {concrete_thickness_c:.2f} cm")


Input values - Nw: 40, d: 2.0, tU: 1.0, tI: 1.0
RtI: 0.8324541127798754

****SCAN ROOM****

Transmission Factor (B) for the Scan room (Uncontrolled area): 0.1586
Transmission Factor (B) for the Scan room (Controlled area): 0.7928 

****UnControl****
Shielding thickness for lead (Uncontrolled Area): 12.94 mm
Shielding thickness for concrete (Uncontrolled Area): 16.31 cm

****Control****
Shielding thickness for lead (Controlled Area): 1.95 mm
Shielding thickness for concrete (Controlled Area): 3.94 cm
